
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

The French mathematician Emile Borel once mused that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of **Recurrent Neural Networks and LSTMs**!

Our goal in this projectis to build a **Shakespeare Sonnet Generator**.<br>
Given a prompt of a few words as input, its task is to generate follow-on text that reads like a Shakespeare Sonnet!<br>

To build our Sonnet Generator we will use a type of model called a **sequence model**. Given a short sequence, a sequence  model predicts the **most likely next item in the sequence**. Sequence models are astonishingly versatile and powerful, because the **sequence** we want to predict can be quite general! It can be composed of **words**, or of **characters**, or of **musical notes**, or of data points in a **time series** such as EKG voltages, or stock prices, or even a sequence of **DNA nucleotides**! 

We will train our model on the entire corpus of Shakespeare's Sonnets, and the model will learn from that data the most likely patterns of characters.

In [ ]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# import a custom text data preparation class
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
from data_cleaning_toolkit_class import data_cleaning_toolkit

'wget' is not recognized as an internal or external command,
operable program or batch file.


### Use `request` to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) to learn how to download the Shakespeare Sonnets from the Gutenberg website. 


In [ ]:
# download all of Shakespeare's Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use requests and the url to download all of the sonnets - save the result to `data`
data = requests.get(url_shakespeare_sonnets)


In [ ]:
# extract the downloaded text from the requests object and save it to `raw_text_data`
# hint: take a look at the attributes of `data`
# YOUR CODE HERE
raw_text_data = data.text


In [ ]:
# check the data type of `raw_text_data`
assert(type(raw_text_data)==str)

### Data Cleaning

In [ ]:
# as usual, we need to clean up the messy data
raw_text_data[:3000]

'\ufeffThe Project Gutenberg eBook of The Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: The Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: September, 1997 [eBook #1041]\r\n[Most recently updated: November 25, 2021]\r\n\r\nLanguage: English\r\n\r\n\r\nProduced by:  the Project Gutenberg Shakespeare Team\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE SONNETS ***\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That t

In [ ]:
# Which characters could we use with the split() method to split the text into lines?

# split the text into **lines** and save the result to `split_data`

# YOUR CODE HERE
split_data = raw_text_data.split('\r\n')
split_data

['\ufeffThe Project Gutenberg eBook of The Sonnets, by William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '',
 'Title: The Sonnets',
 '',
 'Author: William Shakespeare',
 '',
 'Release Date: September, 1997 [eBook #1041]',
 '[Most recently updated: November 25, 2021]',
 '',
 'Language: English',
 '',
 '',
 'Produced by:  the Project Gutenberg Shakespeare Team',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK THE SONNETS ***',
 '',
 '',
 '',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '  I',
 '',
 '  From fairest crea

In [ ]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 
split_data[:80] 

['\ufeffThe Project Gutenberg eBook of The Sonnets, by William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '',
 'Title: The Sonnets',
 '',
 'Author: William Shakespeare',
 '',
 'Release Date: September, 1997 [eBook #1041]',
 '[Most recently updated: November 25, 2021]',
 '',
 'Language: English',
 '',
 '',
 'Produced by:  the Project Gutenberg Shakespeare Team',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK THE SONNETS ***',
 '',
 '',
 '',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '  I',
 '',
 '  From fairest crea

**Use list index slicing to remove the titles and descriptions, so we only have the sonnets.**


In [ ]:
# we need to drop all the boilerplate text (i.e., titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 

# find index boundaries (start, end)so that 
# sonnets exist between these indices 
# titles and descriptions exist outside of these indices

# use index slicing to isolate the sonnet lines from the text - save the result to `sonnets`

# YOUR CODE HERE
start_index = 45
end_index = -369
sonnets = split_data[start_index:end_index]

Notice that there are many lines that should not be counted as sonnets!

In [ ]:
# these non-sonnet lines have far fewer characters than the actual sonnet lines?

sonnets[200:240]

['  Nor lose possession of that fair thou ow’st,',
 '  Nor shall death brag thou wander’st in his shade,',
 '  When in eternal lines to time thou grow’st,',
 '    So long as men can breathe, or eyes can see,',
 '    So long lives this, and this gives life to thee.',
 '',
 '  XIX',
 '',
 '  Devouring Time, blunt thou the lion’s paws,',
 '  And make the earth devour her own sweet brood;',
 '  Pluck the keen teeth from the fierce tiger’s jaws,',
 '  And burn the long-liv’d phoenix, in her blood;',
 '  Make glad and sorry seasons as thou fleets,',
 '  And do whate’er thou wilt, swift-footed Time,',
 '  To the wide world and all her fading sweets;',
 '  But I forbid thee one most heinous crime:',
 '  O! carve not with thy hours my love’s fair brow,',
 '  Nor draw no lines there with thine antique pen;',
 '  Him in thy course untainted do allow',
 '  For beauty’s pattern to succeeding men.',
 '    Yet, do thy worst old Time: despite thy wrong,',
 '    My love shall in my verse ever live youn

In [ ]:
# use your judgement to decide on a good value for  
#   the  minimum number of characters that a sonnet should have
#   call it n_chars
n_chars = 10

# Let's use that observation to filter out all the non-sonnet lines!
#    save results to `filtered_sonnets`
# Hint: use a list comprehension
filtered_sonnets = [line.lstrip('') for line in sonnets if len(line) > n_chars]

# YOUR CODE HERE


In [ ]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets

['    To be death’s conquest and make worms thine heir.',
 '  Lo! in the orient when the gracious light',
 '  Lifts up his burning head, each under eye',
 '  Doth homage to his new-appearing sight,',
 '  Serving with looks his sacred majesty;',
 '  And having climb’d the steep-up heavenly hill,',
 '  Resembling strong youth in his middle age,',
 '  Yet mortal looks adore his beauty still,',
 '  Attending on his golden pilgrimage:',
 '  But when from highmost pitch, with weary car,',
 '  Like feeble age, he reeleth from the day,',
 '  The eyes, ’fore duteous, now converted are',
 '  From his low tract, and look another way:',
 '    So thou, thyself outgoing in thy noon:',
 '    Unlook’d, on diest unless thou get a son.',
 '  Music to hear, why hear’st thou music sadly?',
 '  Sweets with sweets war not, joy delights in joy:',
 '  Why lov’st thou that which thou receiv’st not gladly,',
 '  Or else receiv’st with pleasure thine annoy?',
 '  If the true concord of well-tuned sounds,',
 '  B

### Use Custom Data Cleaning Tool 

Use one of the methods in the `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [ ]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# YOUR CODE HERE
import data_cleaning_toolkit

In [ ]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# YOUR CODE HERE
dctk = data_cleaning_toolkit()
clean_sonnets = [dctk.clean_data(line) for line in filtered_sonnets]

In [ ]:
# much better!
display(clean_sonnets)
print(len(clean_sonnets))

['to be deaths conquest and make worms thine heir',
 'lo in the orient when the gracious light',
 'lifts up his burning head each under eye',
 'doth homage to his newappearing sight',
 'serving with looks his sacred majesty',
 'and having climbd the steepup heavenly hill',
 'resembling strong youth in his middle age',
 'yet mortal looks adore his beauty still',
 'attending on his golden pilgrimage',
 'but when from highmost pitch with weary car',
 'like feeble age he reeleth from the day',
 'the eyes fore duteous now converted are',
 'from his low tract and look another way',
 'so thou thyself outgoing in thy noon',
 'unlookd on diest unless thou get a son',
 'music to hear why hearst thou music sadly',
 'sweets with sweets war not joy delights in joy',
 'why lovst thou that which thou receivst not gladly',
 'or else receivst with pleasure thine annoy',
 'if the true concord of welltuned sounds',
 'by unions married do offend thine ear',
 'they do but sweetly chide thee who confounds',

1491


### Use Your Data Tool to Create Character Sequences 
for the LSTM model

We'll need the `create_char_sequences` method for this task. <br>
However, this method requires a parameter called `maxlen,` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

To answer that question, let's do some statistics! 

In [ ]:
def calc_stats(corpus):
    """
    Calculates statistics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnet's line length - save the results to `doc_lens` 

    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code

    # YOUR CODE HERE
    doc_lens = [len(line) for line in corpus]
    return np.mean(doc_lens),np.median(doc_lens),np.std(doc_lens),np.min(doc_lens),np.max(doc_lens), 

In [ ]:
# sonnet line length statistics 
mean, med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(40.97317236753857, 41.0, 4.022816123127845, 27, 57)

In [ ]:
# from the results of the sonnet line length statistics, use your judgement to select a value for maxlen
#   hint -- a good value might be half the median length of a sonnet line
# use .create_char_sequences() to create sequences

# YOUR CODE HERE
maxlen = 40
dctk.create_char_sequences(clean_sonnets, maxlen=maxlen)

Created 12509 sequences.


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. <br>
Let's call them in the cells below.

In [ ]:
# number of input features for our LSTM model
dctk.n_features

27

In [ ]:
# unique characters that appear in our sonnets 
dctk.unique_chars

['r',
 'x',
 'm',
 'd',
 'k',
 'b',
 'y',
 'l',
 'w',
 'z',
 'h',
 'p',
 'i',
 't',
 'v',
 'c',
 'q',
 'f',
 'u',
 's',
 'g',
 'j',
 ' ',
 'n',
 'a',
 'e',
 'o']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e., **dctk.unique_chars**) and the `number of model input features` (i.e., **dctk.n_features**) the same?

**Hint:** The model that we will shortly build here is very similar to the text generation model we built in the guided project.

**Answer 1:**

They are the same because despite there being many words of varying length, there are only a max of 27 unique characters that make up every sonnet


**Question 2:**

Take a look at the printout of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

Because without the whitespace we wouldnt be able to create words, just letters

----

### Use Our Data Tool to Create X and Y Splits

You'll need the `create_X_and_Y` method for this task. 

In [ ]:
# TODO: provide a walkthrough of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [ ]:
# notice that our input array isn't a matrix - it's a rank three tensor
X.shape

(12509, 40, 27)

In $X$.shape, we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [ ]:
# first index returns a signal sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e., $X[i]$ where $i$ is some index value) is `maxlen` long and <br>
has a number of features equal to `dctk.n_features`. <br>Let's try to understand this shape.

In [ ]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(40, 27)

**Each row corresponds to a character vector,** and there is `maxlen` number of character vectors. 

**Each column corresponds to a unique character,** and there are `dctk.n_features` number of features. 


In [ ]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

Notice that there is a single `True` value, and all the rest of the values are `False`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will be a single `True` value, and the rest will be `False`. 

Let's say that `True` appears in the $ith$ index; by  $ith$ index we mean some index in the general case. So how can we find out which character corresponds to?

To answer this question, we need to use the character-to-integer look-up dictionary. 

In [ ]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that character vector is endcoding for
dctk.int_char

{0: 'r',
 1: 'x',
 2: 'm',
 3: 'd',
 4: 'k',
 5: 'b',
 6: 'y',
 7: 'l',
 8: 'w',
 9: 'z',
 10: 'h',
 11: 'p',
 12: 'i',
 13: 't',
 14: 'v',
 15: 'c',
 16: 'q',
 17: 'f',
 18: 'u',
 19: 's',
 20: 'g',
 21: 'j',
 22: ' ',
 23: 'n',
 24: 'a',
 25: 'e',
 26: 'o'}

In [ ]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

t
o
 
b
e
 
d
e
a
t
h
s
 
c
o
n
q
u
e
s
t
 
a
n
d
 
m
a
k
e
 
w
o
r
m
s
 
t
h
i
Sequence length: 40


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape printout of `X.shape` to a classmate?


**Answer 1:**

When you look at the shape there are 3 different numbers youll see. The first is the length of the total number of sonnets were going to search over, the second number is the length of each instance or sonnet, and the final number is the number of unique values within the sonnets

----


### Build a Shakespeare Sonnet Text Generation Model

Now that we have prepped our data (and understood that process), let's finally build out our character generation model, similar to what we did in the guided project.<br>

First, we'll create a callback to monitor the training -- by printing a sample of text generated by the model at the end of each epoch.

Helper function to generate a sample character:

In [ ]:
def sample(preds, temperature=1.0):
    """
    Helper function to generate a sample character
    Input is a predictions vector from our model, for example a set of 27 character probabilities
    Output is the index of the generated character 
    """
    # convert predictions to an array 
    preds = np.asarray(preds).astype('float64')

    # use the temperature hyper-parameter to "warp" (sharpen or spread out) the probability distribution 
    preds = np.log(preds) / temperature

    # use the softmax activation function to create a new list of probabilities 
    #   corresponding to the "warped" probability distribution
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Draw a single sample from a multinomial distribution, given these probabilities
    #   The sample will be a one-hot encoded character
    """ Notes on the np.random.multinomial() function 
       The first argument is the number of "trials" we want: 1 in this case
       The second argument is the list of probabilities for each character
       The third argument is number of sets of "trials" we want: again, 1 in this case
       By analogy with a dice-rolling experiment: 
          This "trial" consists of generating a single "throw" of a die with 27 faces;
             each face corresponds to a character and its associated probability
    """

    probas = np.random.multinomial(1, preds, 1)
    
    # return the index that corresponds to the max probability 
    return np.argmax(probas)


Create the `on_epoch_end` function to be passed into `LambdaCallback()`

In [ ]:
def on_epoch_end(epoch, _):
    """"
    Function invoked at the end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunece into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next maxlen chars should be that follow the seed string
    for i in range(maxlen):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)
print(f'All of Shakespeare\'s sonnets comprise about {len(text)} characters')

All of Shakespeare's sonnets comprise about 62581 characters


Create the callback object

In [ ]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Build and Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load (in the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [ ]:
# build text generation model layer by layer 
# fit model

# YOUR CODE HERE
model = Sequential()
model.add(LSTM(264,input_shape=(dctk.maxlen,dctk.n_features),return_sequences=True, activation="relu"))
model.add(LSTM(128, activation="relu"))
model.add(Dense(dctk.n_features,activation="softmax"))
model.compile(loss = "categorical_crossentropy",optimizer = "adam")
model.fit(X,y,batch_size =256, epochs=50, callbacks=[print_callback],workers=10)

Epoch 1/50
49/49 [==============================] - ETA: 0s - loss: 3.0551
----- Generating text after Epoch: 0
----- Generating with seed: "character was done that i might see what"
character was done that i might see whattz rcnnisef s ntuonguiuuhh  in uj nrb r 
49/49 [==============================] - 17s 318ms/step - loss: 3.0551
Epoch 2/50
49/49 [==============================] - ETA: 0s - loss: 2.8292
----- Generating text after Epoch: 1
----- Generating with seed: " rare that heavens air in this huge rond"
 rare that heavens air in this huge rondudv  lehbfo   o  m atr foetshehtlw d lhl
49/49 [==============================] - 15s 317ms/step - loss: 2.8292
Epoch 3/50
49/49 [==============================] - ETA: 0s - loss: 2.8225
----- Generating text after Epoch: 2
----- Generating with seed: "love should ever dwell whateer thy thoug"
love should ever dwell whateer thy thougb hruulhmefeuemd narmi theihrpsid ioehup
49/49 [==============================] - 16s 316ms/step - loss: 2.

### Save the trained model to a file

In [ ]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's Play With Our Trained Model 

Now that we have a trained model that, though far from perfect, can generate actual English words, we can look at the predictions to continue learning more about how a text generation model works.

We can also take this as an opportunity to unpack the `def on_epoch_end` function to understand better how it works. 

In [ ]:
# this is our joined clean sonnet data
text

'to be deaths conquest and make worms thine heir lo in the orient when the gracious light lifts up his burning head each under eye doth homage to his newappearing sight serving with looks his sacred majesty and having climbd the steepup heavenly hill resembling strong youth in his middle age yet mortal looks adore his beauty still attending on his golden pilgrimage but when from highmost pitch with weary car like feeble age he reeleth from the day the eyes fore duteous now converted are from his low tract and look another way so thou thyself outgoing in thy noon unlookd on diest unless thou get a son music to hear why hearst thou music sadly sweets with sweets war not joy delights in joy why lovst thou that which thou receivst not gladly or else receivst with pleasure thine annoy if the true concord of welltuned sounds by unions married do offend thine ear they do but sweetly chide thee who confounds in singleness the parts that thou shouldst bear mark how one string sweet husband to a

In [ ]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

34587

In [ ]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e., input sequence into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

In [ ]:
# display the "seed string" i.e. the input sequence into the model
print('----- Input seed: "' + sentence + '"')

----- Input seed: "not so stout nor gates of steel so stron"


In [ ]:
# use model to predict what the next maxlen chars should be that follow the seed string
for i in range(maxlen):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [ ]:
# this is the seed string
generated

'not so stout nor gates of steel so stron'

In [ ]:
# these are the maxlen chars that the model thinks should come after the seed stirng
sentence

'g thee yet if thut anvievthlend show com'

In [ ]:
# how put it all together
generated + sentence

'not so stout nor gates of steel so strong thee yet if thut anvievthlend show com'

# Resources and Stretch Goals

## Stretch Goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g., plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_generation) - code for training an RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem and provides an example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN